# Dobbeltspalte-simulation

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft2, ifft2, fftfreq
import matplotlib.animation as animation
plt.style.use('ggplot')
%matplotlib qt

#### Sæt simulationsbetingelser

In [2]:
N = 2**10 #antal punkter i xgrid og pgrid
dt = 0.02 #Tidsskridt
timesteps = 300 #Antal simulationstrin

#### Initialiser grid

In [3]:
x = np.linspace(-100,100,N) #Lav N*N grid mellem -100 og 100
x1,x2 = np.meshgrid(x,x) #Definer 2d x-gitter som 2 meshgrids
dx = 200/N

p = fftfreq(N,dx)*2*np.pi #Pgrid som i 1d
p1,p2 = np.meshgrid(p,p) #Definer 2d p-gitter som 2 meshgrids

#### Definer $\Psi_0$

In [4]:
def gaus2d(x=0, y=0, mx=0, my=0, sx=20, sy=50):
    return 1. / (2. * np.pi * sx * sy) * np.exp(-((x - mx)**2. / (2. * sx**2.) + (y - my)**2. / (2. * sy**2.)))

#Generer gaussisk startbølgefunktion med impuls:
psi0 = gaus2d(x1, x2, mx=np.mean(x1)-40, my=np.mean(x2)) * np.exp(-1j*(-8)*x)

#### Definer V(x)

In [5]:
d,a=80,30 #Egenskaber ved dobbeltspalten

V = np.zeros_like(x1)
V[0:512-int(d/2)-a,500:520] = 500
V[512-int(d/2):512+int(d/2),500:520] = 500
V[512+int(d/2):512,500:520] = 500
V[512+int(d/2)+a:1024,500:520] = 500

#### Definer operatorer jævnfør teori

In [6]:
def v_operator(psi):
    return psi * np.exp(-1.j * V * dt)

def t_operator(psi):
    return psi * np.exp(-1.j * (p1**2 + p2**2) * dt/2)

#### Kør simulation og populer [psis]

In [7]:
psis = []
psi = psi0
for t in range(timesteps):
    psis.append(np.abs(psi)**2)
    psi = t_operator(fft2(psi))
    psi = v_operator(ifft2(psi))
    psi = t_operator(fft2(psi))
    psi = ifft2(psi)

#### Animer

In [8]:
fig,ax = plt.subplots(1,1)

def update(i):
       ax.clear()
       ax.set_title('Simulated quantum mechanical double slit experiment')
       ax.contourf(x1,x2,psis[i],100)
    
anim = animation.FuncAnimation(fig,
                           update,
                           frames=len(psis),
                           interval=2)
anim

#### (EVT) gem animation til .gif

In [9]:
#writergif = animation.PillowWriter(fps=30)
#anim.save('ds.gif',writer=writergif)